In [26]:
import os
import pandas as pd
import pickle
import json
import requests
import ast

import turicreate
from turicreate import SFrame

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model, InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, LocalWebservice, Webservice,  AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [27]:
print("AZML SDK Version:", azureml.core.VERSION)
print("Pandas Version:", pd.__version__)

AZML SDK Version: 1.48.0
Pandas Version: 1.1.5


In [28]:
# If in Databricks
# OUTPUT_DIR = "/dbfs/FileStore/RetailRecommender/output/"
# MODELS_DIR = "/dbfs/FileStore/RetailRecommender/models/"
# DEPLOY_DIR = "/dbfs/FileStore/RetailRecommender/deploy/"

OUTPUT_DIR = "./output/"
MODELS_DIR = "./models/"
DEPLOY_DIR = "./deploy/"

os.makedirs(DEPLOY_DIR, exist_ok=True)

# 1. Load the AML Workspace

In [29]:
ws = Workspace(workspace_name="RRBrazilex", 
                subscription_id="2f8a2563-4e90-4a88-8e6c-56fa0f12cb7e", 
                resource_group="retail-rec-brazil-ex")

# ws = Workspace(workspace_name="YOUR_WORKSPACE_NAME", 
#                 subscription_id="YOUR_SUBSCRIPTION_ID", 
#                 resource_group="YOUR_RESOURCE_GROUP")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = "\n")

RRBrazilex
retail-rec-brazil-ex
eastus
2f8a2563-4e90-4a88-8e6c-56fa0f12cb7e


# 2. Register the Models on AZML service model registry

In [30]:
model_s_path = MODELS_DIR + "recommendation_s.model"
model_r_path = MODELS_DIR + "recommendation_r.model"

In [31]:
azmodel_s = Model.register( model_path = model_s_path,
                           model_name = "similarity_model",
                           workspace = ws)

azmodel_r = Model.register( model_path = model_r_path,
                           model_name = "ranking_factorization_model",
                           workspace = ws)

Registering model similarity_model
Registering model ranking_factorization_model


# 3. Test score.py init() and run() functions

In [33]:
import os

def init():
    
    global model_s
    global model_r
    global product_column_name
    global customer_column_name
    global reviewscore_column_name
    
    order_column_name = "order_id"
    product_column_name = "product_id"
    timestamp_column_name = "order_purchase_timestamp"
    category_column_name = "product_category_name_english"
    reviewscore_column_name = "review_score"
    customer_column_name = "customer_unique_id"
    customer_city_column_name = "customer_city"
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    

In [34]:
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if customer_column_name in data:
            print(customer_column_name + " found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model\n")
                users_query = turicreate.SFrame(
                    {"visitorid": data[customer_column_name], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data[customer_column_name]),
                    "month": data["month"] if "month" in data else [8] * len(data[customer_column_name]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data[customer_column_name])
                    }
                )
                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model\n")
                result = model_s.recommend(users=data[customer_column_name], k=10).to_dataframe()
                
            if product_column_name in data:
                return "Error: Request cannot have" + product_column_name + " and" + customer_column_name + " together. Please make separate requests for user recommendations and for items basket similarity"
        
        elif product_column_name in data:
                print("Using Similarity Model\n")
                result = model_s.get_similar_items(data[product_column_name], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named " + customer_column_name + " or " + product_column_name
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e


In [35]:
init()

In [36]:

test_load1 = '''
    [
        {"visitorid":6000, "period":"Night",  "weekday":2},
        {"visitorid":871766c5855e863f6eccc05f988b23cb, "period":"Night", "weekday":2}
    ]
'''

test_load2 = '''
    [
        {"visitorid":6000},
        {"visitorid":871766c5855e863f6eccc05f988b23cb}
    ]
'''

test_load3 = '''
    [
        {"visitorid":6000,"period":"Night",  "weekday":2 , "month": 9},
        {"visitorid":871766c5855e863f6eccc05f988b23cb,"period":"Night",  "weekday":2 , "month": 9}
    ]
'''

test_load4 = '''
    [
        {"product_id":4244733e06e7ecb4970a6e2683c13e61},
        {"product_id":fdcf45aa23bb8312ecc0027d6e1ef1c4}
    ]
'''


In [37]:
run(test_load3)

JSONDecodeError: Expecting ',' delimiter: line 4 column 28 (char 106)

In [17]:
# run(test_load4)

# 4. Create Inference Environment

In [18]:
# curated_env = Environment.get(workspace=ws, name="AzureML-minimal-ubuntu18.04-py37-cpu-inference")
# myenv = curated_env.clone("turicreate_env")

conda_dep = CondaDependencies()
conda_dep.add_pip_package("turicreate")

myenv = Environment(name="turicreate_env")
myenv.python.conda_dependencies=conda_dep
myenv.inferencing_stack_version = "latest"

# Register the Environment on AZML so we can retreive it later
myenv = myenv.register(workspace=ws)

# Uncomment this to re-build the environment
# myenv.build(workspace=ws)


# 5. Create the inference configuration

### !!! Make sure that in score.py you set the right model version!!!

In [19]:
scorepy_text = '''
import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "{S_VERSION}"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "{R_VERSION}"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "{USER_ID}" in data:
            print("{USER_ID} found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model")
                users_query = turicreate.SFrame(
                    {{"visitorid": data["{USER_ID}"], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data["{USER_ID}"]),
                    "month": data["month"] if "month" in data else [8] * len(data["{USER_ID}"]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data["{USER_ID}"])
                    }}
                )
                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model")
                result = model_s.recommend(users=data["{USER_ID}"], k=10).to_dataframe()
                
            if "{ITEM_ID}" in data:
                return "Error: Request cannot have {USER_ID} and {ITEM_ID} together. Please make separate requests for {USER_ID} recommendations and for {ITEM_ID} basket similarity"
        
        elif "{ITEM_ID}" in data:
                print("Using Similarity Model")
                result = model_s.get_similar_items(data["{ITEM_ID}"], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named {USER_ID} or {ITEM_ID}"
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e

'''.format(S_VERSION=1, R_VERSION=1, USER_ID=customer_column_name, ITEM_ID=product_column_name)

with open(DEPLOY_DIR+"score.py", "w") as file:
    file.write(scorepy_text)

print("score.py written, using",DEPLOY_DIR+"score.py")

score.py written, using ./deploy/score.py


In [20]:
!cat {DEPLOY_DIR}score.py


import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "1"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "1"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "customer_unique_id" in data:
      

# 6 . Deploy to Local
This does not work on Databricks

In [21]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [azmodel_s, azmodel_r], inference_config, deployment_config)

/tmp/ipykernel_64236/3062853723.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [azmodel_s, azmodel_r], inference_config, deployment_config)


Generating Docker build context.
2023/02/01 02:04:33 Downloading source code...
2023/02/01 02:04:34 Finished downloading source code
2023/02/01 02:04:34 Creating Docker network: acb_default_network, driver: 'bridge'
2023/02/01 02:04:35 Successfully set up Docker network: acb_default_network
2023/02/01 02:04:35 Setting up Docker configuration...
2023/02/01 02:04:35 Successfully set up Docker configuration
2023/02/01 02:04:35 Logging in to registry: 8cd100591dc0495bba6b05e638847d61.azurecr.io
2023/02/01 02:04:36 Successfully logged into 8cd100591dc0495bba6b05e638847d61.azurecr.io
2023/02/01 02:04:36 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/02/01 02:04:36 Scanning for dependencies...
2023/02/01 02:04:36 Successfully scanned dependencies
2023/02/01 02:04:36 Launching container with name: acb_step_0
Sending build context to Docker daemon  72.19kB
Step 1/28 : FROM mcr.microsoft.com/azureml/o16n-base/python-assets:latest AS 

In [52]:
# If container fails, check out the logs
# local_service.get_logs()

## Test local web service container

In [24]:
print("Local service hosted in:", local_service.scoring_uri)

Local service hosted in: http://localhost:6789/score


In [25]:
import json

response = local_service.run(input_data = test_load3)
print(response)

Checking container health...
Local webservice is running at http://localhost:6789


Error: Scoring was unsuccessful.



WebserviceException: WebserviceException:
	Message: Error: Scoring was unsuccessful.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Scoring was unsuccessful."
    }
}

In [55]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = local_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"visitorid":6000,"itemid":64700,"score":0.0598958333,"rank":1},{"visitorid":6000,"itemid":158457,"score":0.0555555622,"rank":2},{"visitorid":6000,"itemid":54572,"score":0.0476190448,"rank":3},{"visitorid":6000,"itemid":342866,"score":0.0476190448,"rank":4},{"visitorid":6000,"itemid":393111,"score":0.0476190448,"rank":5},{"visitorid":6000,"itemid":464114,"score":0.0476190448,"rank":6},{"visitorid":6000,"itemid":50575,"score":0.0476190448,"rank":7},{"visitorid":6000,"itemid":279324,"score":0.0416666667,"rank":8},{"visitorid":6000,"itemid":342139,"score":0.0416666667,"rank":9},{"visitorid":6000,"itemid":26634,"score":0.0388888915,"rank":10},{"visitorid":567987987988,"itemid":456056,"score":0.0257491243,"rank":1},{"visitorid":567987987988,"itemid":119736,"score":0.0251864088,"rank":2},{"visitorid":567987987988,"itemid":338395,"score":0.0233529449,"rank":3},{"visitorid":567987987988,"itemid":439963,"score":0.0216234672,"rank":4},{"visitorid":567987987988,"itemid":186702,"score":0.02007171

# 7. Deploy to Azure Container Instance

In [56]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

aci_service = Model.deploy(ws, "aci-service", [azmodel_s, azmodel_r], inference_config, deployment_config, overwrite=True)

aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-07 00:04:35+00:00 Creating Container Registry if not exists.
2022-12-07 00:04:35+00:00 Registering the environment.
2022-12-07 00:04:37+00:00 Use the existing image.
2022-12-07 00:04:37+00:00 Generating deployment configuration.
2022-12-07 00:04:40+00:00 Submitting deployment to compute.
2022-12-07 00:04:43+00:00 Checking the status of deployment test..
2022-12-07 00:09:51+00:00 Checking the status of inference endpoint test.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test ACI URL API Endpoint

In [58]:
aci_service.scoring_uri

'http://71f08952-e588-491d-a93e-40ce53b4cff3.southcentralus.azurecontainer.io/score'

In [57]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = aci_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"visitorid":6000,"itemid":64700,"score":0.0598958333,"rank":1},{"visitorid":6000,"itemid":158457,"score":0.0555555622,"rank":2},{"visitorid":6000,"itemid":54572,"score":0.0476190448,"rank":3},{"visitorid":6000,"itemid":342866,"score":0.0476190448,"rank":4},{"visitorid":6000,"itemid":393111,"score":0.0476190448,"rank":5},{"visitorid":6000,"itemid":464114,"score":0.0476190448,"rank":6},{"visitorid":6000,"itemid":50575,"score":0.0476190448,"rank":7},{"visitorid":6000,"itemid":279324,"score":0.0416666667,"rank":8},{"visitorid":6000,"itemid":342139,"score":0.0416666667,"rank":9},{"visitorid":6000,"itemid":26634,"score":0.0388888915,"rank":10},{"visitorid":567987987988,"itemid":456056,"score":0.0257491243,"rank":1},{"visitorid":567987987988,"itemid":119736,"score":0.0251864088,"rank":2},{"visitorid":567987987988,"itemid":338395,"score":0.0233529449,"rank":3},{"visitorid":567987987988,"itemid":439963,"score":0.0216234672,"rank":4},{"visitorid":567987987988,"itemid":186702,"score":0.02007171

In [59]:
aci_service.delete()

# 8. Deploy to Production -  managed AKS Cluster

Details for SSL and more best practices for production here: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.ipynb

In [60]:
# Choose a name for your AKS cluster
aks_name = 'my-aks'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    # https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.akscompute?view=azure-ml-py
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [63]:
# Set the web service configuration (using default here)
# https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#azureml-core-webservice-akswebservice-deploy-configuration
aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

In [64]:
%%time

aks_service_name ='aks-service'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[azmodel_s, azmodel_r],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-07 00:11:49+00:00 Creating Container Registry if not exists.
2022-12-07 00:11:49+00:00 Registering the environment.
2022-12-07 00:11:50+00:00 Use the existing image.
2022-12-07 00:11:54+00:00 Checking the status of deployment aks-service..
2022-12-07 00:12:25+00:00 Checking the status of inference endpoint aks-service.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 373 ms, sys: 86.9 ms, total: 460 ms
Wall time: 42.9 s


# 8. Test the web service using raw HTTP request

In [65]:
# # if (key) auth is enabled, retrieve the API keys. AML generates two keys.
key1, Key2 = aks_service.get_keys()
print(key1)

# # if token auth is enabled, retrieve the token.
# access_token, refresh_after = aks_service.get_token()

G46wSFtiiAFlCHHi0beyU29ceaB2dPRS


In [66]:
print(aks_service.scoring_uri)

http://20.114.104.171:80/api/v1/service/aks-service/score


In [68]:
apiurl = aks_service.scoring_uri
key = key1
# key = "G46wSFtiiAFlCHHi0beyU29ceaB2dPRS"
# apiurl = "http://20.114.104.171:80/api/v1/service/aks-service/score"

In [69]:
%%time

# # If (key) auth is enabled, don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key}

# # If token auth is enabled, don't forget to add token to the HTTP header.
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

resp = requests.post(apiurl, bytes(test_load1,encoding = 'utf8'), headers=headers)

print(ast.literal_eval(resp.text))

[{"visitorid":6000,"itemid":119736,"score":1.2709273953,"rank":1},{"visitorid":6000,"itemid":32227,"score":1.2562527951,"rank":2},{"visitorid":6000,"itemid":234215,"score":1.2536956422,"rank":3},{"visitorid":6000,"itemid":210002,"score":1.2502753945,"rank":4},{"visitorid":6000,"itemid":23762,"score":1.2486156024,"rank":5},{"visitorid":6000,"itemid":76196,"score":1.2428713669,"rank":6},{"visitorid":6000,"itemid":212006,"score":1.2396474962,"rank":7},{"visitorid":6000,"itemid":256721,"score":1.2389497782,"rank":8},{"visitorid":6000,"itemid":123664,"score":1.2384240537,"rank":9},{"visitorid":6000,"itemid":317192,"score":1.2381217134,"rank":10},{"visitorid":567987987988,"itemid":119736,"score":1.6954194245,"rank":1},{"visitorid":567987987988,"itemid":32227,"score":1.692107864,"rank":2},{"visitorid":567987987988,"itemid":234215,"score":1.6887668235,"rank":3},{"visitorid":567987987988,"itemid":76196,"score":1.6697236534,"rank":4},{"visitorid":567987987988,"itemid":210002,"score":1.669452417,

In [70]:
aks_service.delete()

In [ ]:
# aks_service.get_logs()